In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine


In [3]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')


# ROI маркетинг по формуле (ROI=revenue(iap + ad) / spend) исключая органику

In [6]:
# Общий ROI по формуле (ROI=revenue(iap + ad) / spend) исключая органику за период  с '2022-04-01' по '2022-05-01'таблица .geo_daily_report

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi = pd.read_sql(f'''
SELECT
    
    (SUM(af_ad_revenue_usd) + SUM(af_purchase_usd)) /SUM(total_cost) AS roi
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику
ORDER BY 1
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi.head(2)

execution time: 0:00:00.192073


,roi
0,0.444624


In [19]:
# Проверим ROI = LTV * CPI
0.418292 * 1.19

0.49776748

In [20]:
# Общий ROI по формуле (ROI=revenue(iap + ad) / spend) по дням 

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_days = pd.read_sql(f'''
SELECT
    event_date::date,
    (SUM(af_ad_revenue_usd) + SUM(af_purchase_usd)) /SUM(total_cost) AS roi
FROM {TABLE_SCHEMA}.geo_daily_report 
WHERE
    1=1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику
GROUP BY 1
ORDER BY 1
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_days.head(2)

execution time: 0:00:00.193073


,event_date,roi
0,2022-04-01,0.472950
1,2022-04-02,0.821768


In [22]:
# Общий ROI по формуле (ROI=revenue(iap + ad) / spend) по дням 

x = roi_days.event_date              
y1 = roi_days.roi      

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y1,  name='roi', mode='lines+markers',))

fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  title="Общий ROI по формуле (ROI=event_date(iap + ad) / spend) по дням",
                  margin=dict(l=0, r=0, t=30, b=0))
fig.update_traces(hoverinfo="all", hovertemplate="event_date: %{x}<br>roi: %{y}")
fig.show()

***Видим резкий взлет  4 апреля и  24 мая***

# Гипотеза рост за счет рекламной кампании
Посмотрим на события за эти даты

In [12]:

data_start ='2022-04-03'
data_end ='2022-04-05'
data_project_id = 'MiT'

geo_daily_report_sql = pd.read_sql(f'''
SELECT 
    event_date,
    country_code,
    media_source,
    campaign,
    impressions,
    clicks,
    installs,
    total_cost,
    af_purchase_usd,
    af_ad_revenue_usd
FROM {TABLE_SCHEMA}.geo_daily_report       
WHERE 
    1 = 1
    AND project_id = '{data_project_id}' 
    AND event_date >='{data_start}'
    AND event_date < '{data_end}'
    AND media_source != 'Organic' --исключая органику
ORDER BY af_ad_revenue_usd DESC
''', con=ENGINE)
geo_daily_report_sql.head(3)

,event_date,country_code,media_source,campaign,impressions,clicks,installs,total_cost,af_purchase_usd,af_ad_revenue_usd
0,2022-04-03,US,bytedanceglobal_int,MiT_EM_AND_US_Purchase_IT1_2703,14615.0,257.0,100,99.64,52.707900,9.241499
1,2022-04-04,US,bytedanceglobal_int,MiT_EM_AND_US_Purchase_IT1_2703,11.0,NaN,31,1.85,13.196790,7.023296
2,2022-04-03,MY,bytedanceglobal_int,MiT_EM_AND_SEA_MAI_IT1_2803,53939.0,2076.0,514,49.98,39.677935,2.771907


***Гипотеза рост за счет рекламной кампании подтвердилась***

In [16]:
# Рассчитаем roi с учетом iap по media_source за период  с '2022-04-01' по '2022-05-01'

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_media_source_iap = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_ad_revenue_usd END AS af_ad_rev_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_ad_revenue_usd END AS af_ad_rev_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_ad_revenue_usd END AS af_ad_rev_Facebook,
            CASE WHEN media_source = 'bytedanceglobal_int' THEN af_purchase_usd END AS af_purchase_usd_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN af_purchase_usd END AS af_purchase_usd_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN af_purchase_usd END AS af_purchase_usd_Facebook,
            CASE WHEN media_source = 'bytedanceglobal_int' THEN total_cost END AS total_cost_bytedanceglobal_int,
            CASE WHEN media_source = 'googleadwords_int' THEN total_cost END AS total_cost_googleadwords_int,
            CASE WHEN media_source = 'Facebook Ads' THEN total_cost END AS total_cost_Facebook
        FROM {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
            AND media_source != 'Organic' --исключая органику
)
SELECT
    --event_date::date,
    (SUM(af_ad_rev_bytedanceglobal_int) + SUM(af_purchase_usd_bytedanceglobal_int)) /SUM(total_cost_bytedanceglobal_int) AS roi_bytedanceglobal,
    (SUM(af_ad_rev_googleadwords_int) + SUM(af_purchase_usd_googleadwords_int)) /SUM(total_cost_googleadwords_int) AS roi_googleadwords,
    (SUM(af_ad_rev_Facebook) + SUM(af_purchase_usd_Facebook)) /SUM(total_cost_Facebook) AS roi_facebook
FROM 
    user_types 
--GROUP BY 1


''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_media_source_iap.head(2)

execution time: 0:00:00.191074


,roi_bytedanceglobal,roi_googleadwords,roi_facebook
0,0.484796,None,0.209511


In [17]:
# Рассчитаем roi сучетом iap по country_code за период  с '2022-04-01' по '2022-05-01'

start_time = datetime.now()

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

roi_country_code = pd.read_sql(f'''
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN country_code = 'US' THEN af_ad_revenue_usd END AS af_ad_rev_us,
            CASE WHEN country_code = 'MY' THEN af_ad_revenue_usd END AS af_ad_rev_my,
            CASE WHEN country_code = 'PH' THEN af_ad_revenue_usd END AS af_ad_rev_ph,
            CASE WHEN country_code = 'UK' THEN af_ad_revenue_usd END AS af_ad_rev_uk,
            CASE WHEN country_code = 'ID' THEN af_ad_revenue_usd END AS af_ad_rev_id,            
            CASE WHEN country_code = 'US' THEN af_purchase_usd END AS af_purchase_us,
            CASE WHEN country_code = 'MY' THEN af_purchase_usd END AS af_purchase_my,
            CASE WHEN country_code = 'PH' THEN af_purchase_usd END AS af_purchase_ph,
            CASE WHEN country_code = 'UK' THEN af_purchase_usd END AS af_purchase_uk,
            CASE WHEN country_code = 'ID' THEN af_purchase_usd END AS af_purchase_id,
            CASE WHEN country_code = 'US' THEN total_cost END AS total_cost_us,
            CASE WHEN country_code = 'MY' THEN total_cost END AS total_cost_my,
            CASE WHEN country_code = 'PH' THEN total_cost END AS total_cost_ph,
            CASE WHEN country_code = 'UK' THEN total_cost END AS total_cost_uk, 
            CASE WHEN country_code = 'ID' THEN total_cost END AS total_cost_id 
        FROM 
            {TABLE_SCHEMA}.geo_daily_report
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND event_date >='{data_start}'
            AND event_date < '{data_end}'
            AND media_source != 'Organic' --исключая органику
)
SELECT
    (SUM(af_ad_rev_us) + SUM(af_purchase_us)) /SUM(total_cost_us) AS roi_us,
    (SUM(af_ad_rev_my) + SUM(af_purchase_my)) /SUM(total_cost) AS roi_my,
    (SUM(af_ad_rev_ph) + SUM(af_purchase_ph)) /SUM(total_cost_ph) AS roi_ph,
    (SUM(af_ad_rev_uk) + SUM(af_purchase_uk)) /SUM(total_cost_uk) AS roi_uk,
    (SUM(af_ad_rev_id) + SUM(af_purchase_id)) /SUM(total_cost_id) AS roi_id
FROM 
    user_types 



''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
roi_country_code.head(2)

execution time: 0:00:00.191072


,roi_us,roi_my,roi_ph,roi_uk,roi_id
0,0.468715,0.015043,None,0.35748,None
